<a href="https://colab.research.google.com/github/hamish-haggerty/AI-hacking/blob/master/SSL/cancer_validation_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cancer_validation_ensemble

> Purpose of this notebook is simply to compare ensembling under two schemes: pretrain=True and pretrain=False. We use BT initial weights to simplify things. Hypothesis: pretraining with BT may help ensemble performance. Why? Because the representations will still be very similar, but a bit different due to randomness.

In [ ]:
#| default_exp cancer_validation_ensemble

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [2]:
#| hide

import os
from google.colab import drive

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():

    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')

    os.chdir('cancer-proj')
    
    os.system('pip install .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

Mounted at /content/drive


In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *
from cancer_proj.cancer_maintrain import *

## Load the data

In [5]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [6]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

## Optionally, display:

In [ ]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [ ]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [7]:
#| export

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

Need to run a few exploratory experiments. Based on the results, next is to run some systematic experiments, probably with W and B... Or final results...

In [8]:
#| export

@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tuner to use with bt initial weights"
    
    self.freeze() #freeze the resnet
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze() #unfreeze the resnet. 
    #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

    self.unfreeze() #We can unfreeze at the end

In [9]:
main_train??

## Test: make sure that it freezes the appropriate part of model:

In [10]:
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
model = LM(encoder)
test_eq(len(my_splitter(model)),2)
test_eq(len(my_splitter_bt(bt_model)),2)

learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.freeze()
print('body should be frozen, (sans batchnorm) linear head unfrozen')
learn.summary()

Downloading: "https://github.com/facebookresearch/barlowtwins/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/barlowtwins/ep1000_bs2048_lrw0.2_lrb0.0048_lambd0.0051/resnet50.pth" to /root/.cache/torch/hub/checkpoints/resnet50.pth


  0%|          | 0.00/90.0M [00:00<?, ?B/s]

body should be frozen, (sans batchnorm) linear head unfrozen


LM (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                               

In [11]:
#| hide

#test : manual. BT

learn = Learner(dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
learn.freeze()
print('body should be frozen, (sans batchnorm) projector unfrozen')
learn.summary()

body should be frozen, (sans batchnorm) projector unfrozen


BarlowTwinsModel (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                 

## Experiment: we want to compare ensemble peformance of two schemes: pretrain=True, and pretrain=False. 

The basic idea is that if we do a small amount of pretraining, it shouldn't destroy the representation, (may even help in fact), but might help increase diversity. One thought that occurs to me: we might want to just have one projector layer, as we don't want to aligh the rep with the projectors? Either that or train them for longer?

In [ ]:
#main_train??

In [52]:
def run_main_train(pretrain,initial_weights='bt_pretrain',num_epochs=10,freeze_num_epochs=5,freeze_numfit=6,numfit=50,num=5):
    "run main_train num times."

    main_dict = {}
    for i in range(num):

        main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
                aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
                initial_weights=initial_weights,pretrain=pretrain,
                num_epochs=num_epochs,numfit=numfit,freeze_num_epochs=freeze_num_epochs,freeze_numfit=freeze_numfit,
                print_report=True,
                        )
        
        metrics = main()
        main_dict[i] = metrics

    return main_dict
        

In [53]:
lst = [True]
results={True:None,False:None}  

for pretrain in lst:
    main_dict = run_main_train(pretrain=pretrain,num=3,num_epochs=50,freeze_num_epochs=10)
    results[pretrain] = main_dict #main_dict has result of running main num=3 times.

#save_dict_to_gdrive('results','baseline_ensemble_results')

Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


epoch,train_loss,valid_loss,time
0,6173.111816,None,00:07
1,5687.053711,None,00:07
2,4814.015137,None,00:06
3,4182.451660,None,00:06
4,3730.583496,None,00:06
5,3376.739990,None,00:07
6,3102.976074,None,00:07
7,2867.964844,None,00:07
8,2664.138672,None,00:06
9,2495.390625,None,00:06


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,1094.539795,None,00:07
1,1053.557617,None,00:06
2,1017.170105,None,00:07
3,995.914612,None,00:07
4,977.820190,None,00:06
5,950.332520,None,00:07
6,934.459290,None,00:07
7,905.342041,None,00:06
8,882.276611,None,00:06
9,853.326355,None,00:06


epoch,train_loss,valid_loss,time
0,2.197326,None,00:06
1,2.194866,None,00:06
2,2.190403,None,00:07
3,2.181928,None,00:06
4,2.167789,None,00:06
5,2.149781,None,00:06


...

/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,1110.988159,None,00:06
1,1138.764526,None,00:06
2,1096.306274,None,00:06
3,1072.618042,None,00:07
4,1040.556274,None,00:06
5,1012.125671,None,00:06
6,978.739685,None,00:07
7,950.338806,None,00:06
8,924.657715,None,00:07
9,897.730286,None,00:07


epoch,train_loss,valid_loss,time
0,2.194930,None,00:07
1,2.193312,None,00:06
2,2.187381,None,00:07
3,2.178009,None,00:06
4,2.163565,None,00:06
5,2.144823,None,00:06


epoch,train_loss,valid_loss,time
0,2.016895,None,00:06
1,2.005534,None,00:06
2,1.997434,None,00:07
3,1.986243,None,00:06
4,1.974843,None,00:06
5,1.959311,None,00:06
6,1.944329,None,00:06
7,1.926249,None,00:06
8,1.902814,None,00:07
9,1.875532,None,00:06


                            precision    recall  f1-score   support

         actinic keratosis       0.65      0.65      0.65        20
      basal cell carcinoma       0.61      0.70      0.65        20
            dermatofibroma       0.76      0.84      0.80        19
                  melanoma       0.53      0.45      0.49        20
                     nevus       0.47      0.45      0.46        20
pigmented benign keratosis       0.50      0.60      0.55        20
      seborrheic keratosis       0.47      0.47      0.47        15
   squamous cell carcinoma       0.71      0.50      0.59        20
           vascular lesion       0.90      0.95      0.93        20

                  accuracy                           0.63       174
                 macro avg       0.62      0.62      0.62       174
              weighted avg       0.63      0.63      0.62       174



#Print out result of each run and save: 

In [54]:
from statistics import mean,stdev
for pretrain in lst:

    _results = [results[pretrain][i]['acc'] for i in range(len(results[pretrain]))]

    print(f"\npretrain={pretrain}, results are: {_results}")
    print(f"mean is: {mean(_results)}\n")
    #print(f"std is: {stdev(_results)}")



pretrain=True, results are: [0.6149425506591797, 0.5632184147834778, 0.6264367699623108]
mean is: 0.6015325784683228



In [ ]:
#save_dict_to_gdrive(results,'base_results')

In [55]:
from itertools import combinations

def within_ensemble_results(results,key,param='pretrain'):

    print(f'Results for {param}={key}')

    _results = list(results[key].values())
    _results = list(combinations(_results,2)) #all pairs of results. So for num=3, will be 3
    for v in _results:

        print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
        print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
        _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
        print(f'Acc of ensemble is:{acc}\n')

In [56]:
within_ensemble_results(results,key=True)

Results for pretrain=True

Acc of first guy in ensemble is: 0.6149425506591797
Acc of second guy in ensemble is: 0.5632184147834778
Acc of ensemble is:0.6034482717514038


Acc of first guy in ensemble is: 0.6149425506591797
Acc of second guy in ensemble is: 0.6264367699623108
Acc of ensemble is:0.5977011322975159


Acc of first guy in ensemble is: 0.5632184147834778
Acc of second guy in ensemble is: 0.6264367699623108
Acc of ensemble is:0.6206896305084229



In [57]:
#within_ensemble_results(results,key=False)

## Conclusion: At least with the settings above (num_epochs=50, freeze_num_epochs=10), performance individually seemed to go down, and ensemble performance didn't increase.

So we can probably scrap this idea (i.e. pretrain: doesn't help individually, or help ensembles)

In [50]:
import itertools
def between_ensemble_results(results,key1,key2,param='pretrain'):

    _results = list(itertools.product(results[key1].values(),results[key2].values()))

    for v in _results:

        print(f"\nAcc of first guy in ensemble ({key1}) is: {v[0]['acc']}")
        print(f"Acc of second guy in ensemble ({key2}) is: {v[1]['acc']}")
        _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
        print(f'Acc of ensemble is:{acc}\n')


 

Between ensemble doesn't matter as much here: i.e. between is between pretrain=True and pretrain=False

In [51]:
between_ensemble_results(results,key1=True,key2=False)


Acc of first guy in ensemble (True) is: 0.6494252681732178
Acc of second guy in ensemble (False) is: 0.6206896305084229
Acc of ensemble is:0.6379310488700867


Acc of first guy in ensemble (True) is: 0.6494252681732178
Acc of second guy in ensemble (False) is: 0.6551724076271057
Acc of ensemble is:0.6551724076271057


Acc of first guy in ensemble (True) is: 0.6494252681732178
Acc of second guy in ensemble (False) is: 0.6264367699623108
Acc of ensemble is:0.6494252681732178


Acc of first guy in ensemble (True) is: 0.6091954112052917
Acc of second guy in ensemble (False) is: 0.6206896305084229
Acc of ensemble is:0.6321839094161987


Acc of first guy in ensemble (True) is: 0.6091954112052917
Acc of second guy in ensemble (False) is: 0.6551724076271057
Acc of ensemble is:0.6551724076271057


Acc of first guy in ensemble (True) is: 0.6091954112052917
Acc of second guy in ensemble (False) is: 0.6264367699623108
Acc of ensemble is:0.6264367699623108


Acc of first guy in ensemble (True) is:

In [ ]:
#| hide

# #old supervised baseline (with fine tune)

# tem = {0: 0.6724137663841248,
#  1: 0.7126436829566956,
#  2: 0.6724137663841248,
#  3: 0.6321839094161987,
#  4: 0.6896551847457886}

# from statistics import mean
# mean(list(tem.values()))